What the data looks like 

for each episode there is a json-file with following structure

{"results":[{"alternatives":[{"transcript":"Hi, this is transcript of first segment","confidence":0.91,"words":[{
                     "startTime":"0s",
                     "endTime":"0.300s",
                     "word":"Hi,"
                  },
                  {"startTime":"0.500s",
                     "endTime":"0.900s",
                     "word":"this"}]}]}]}




What I want the data to look like (kind of)

for each show there shoud be one dict containing show id, show name and all episodes of show split into segments

[{'show-id':'123','show-name':'alex & sigge','episodes':[{'name':'first episode','date':'2020-01-01','segmentTranscripts':[{'segNo':0,'startTime':'00:00s','endTime':'00:20s','transcript':'......',},{},]}]},{},]

- show name would have to be looked up in the metadata.tsv

In [ ]:
import json

def extract_segments(path):
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        #make sure there is only one dict in this list
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            seg_result["transcript"]=segment_dict["transcript"]
            segments.append(seg_result)
            iter+=1

    return segments





In [81]:
import os

base_dir="/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts"
data=[]
for sub_dir in os.listdir(base_dir):
    for sub_sub_dir in os.listdir(base_dir+'/'+sub_dir):
        for show_dir in os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir):
            show_dict={}
            show_dict["showId"]=show_dir
            show_dict["episodes"]=[]
            for i,episode in enumerate(os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir)):
                episode_dict={}
                #remove .json ending
                episode_dict["episodeId"]=episode[:-5]
                episode_dict["episodeNo"]=i
                episode_path=base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir+'/'+episode
                episode_dict["episodeSegments"]=extract_segments(episode_path)
                show_dict["episodes"].append(episode_dict)
            data.append(show_dict)
with open("test.json", "w+") as write_file:
    json.dump(data, write_file)